# Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import os, numba
import time
from tensorflow.keras import mixed_precision

# Training dataset

In [ ]:
base_path = '/Users/jc/Documents/Snl4_data/Data_train/ERA5/Data/'
data_input = pd.read_csv(f'{base_path}input.csv', header=None) 
data_input_dia = pd.read_csv(f'{base_path}input_dia.csv', header=None)
data_output = pd.read_csv(f'{base_path}output.csv', header=None)

In [ ]:
data_in0 = data_input.to_numpy()
data_in_dia0 = data_input_dia.to_numpy()
data_out0 = data_output.to_numpy()
variables = {
    'data_in': data_in0,
    'data_in_dia': data_in_dia0,
    'data_out': data_out0,
}
num_locations = 9
num_years = 8
year_points = [2928 if i in [0, 4] else 2920 for i in range(num_years)]  # Points per year
total_points_per_location = sum(year_points)
data_loc0 = np.concatenate([np.ones(total_points_per_location, dtype=int) * loc for loc in range(1, num_locations + 1)]).reshape(-1, 1)
train_split = 0.9  
def split_by_location(data):
    train_data, test_data = [], []
    for loc_idx in range(num_locations):
        start_idx = loc_idx * total_points_per_location
        end_idx = start_idx + total_points_per_location
        loc_data = data[start_idx:end_idx, :]
        trainn = int(len(loc_data) * train_split)
        train_data.append(loc_data[:trainn, :])
        test_data.append(loc_data[trainn:, :])
    train_combined = np.concatenate(train_data, axis=0)
    train_indices = np.random.RandomState(seed=42).permutation(len(train_combined))
    train_combined = train_combined[train_indices] 
    return train_combined, np.concatenate(test_data, axis=0)
for var_name, var_data in variables.items():
    globals()[f"{var_name}_train"], globals()[f"{var_name}_test"] = split_by_location(var_data)

def reshape_and_transpose(x):
    return x.reshape(x.shape[0], 40, 24).transpose(0, 2, 1)[:, :, :40]
data_in1_train     = reshape_and_transpose(data_in_train)
data_in1_dia_train = reshape_and_transpose(data_in_dia_train)
data_in1_dia_test  = reshape_and_transpose(data_in_dia_test)
data_in1_test      = reshape_and_transpose(data_in_test)
data_out1_train    = reshape_and_transpose(data_out_train)
data_out1_test     = reshape_and_transpose(data_out_test)
data_input_test      = reshape_and_transpose(data_input.to_numpy())
data_input_dia_test  = reshape_and_transpose(data_input_dia.to_numpy())
data_output_test     = reshape_and_transpose(data_output.to_numpy())

print(f"Training_size: {data_input_test.shape}")
print(f"Validation_size: {data_out1_test.shape}")

## Spectrum input only

In [ ]:
data_in1_trainsk = np.stack([data_in1_train], axis=-1) #normalized by max value
data_in1_testsk = np.stack([data_in1_test], axis=-1) 
np.random.seed(42)
random_indices = np.random.choice(data_in1_trainsk.shape[0], size=data_in1_trainsk.shape[0], replace=False)
selected_data_in1x = data_in1_trainsk[random_indices, :, :40]
selected_data_out1x = data_out1_train[random_indices, :, :40]
spec_norm1 = np.std(selected_data_in1x)
snl_norm1 = np.std(selected_data_out1x)
selected_data_in1n = selected_data_in1x/spec_norm1
selected_data_out1n = selected_data_out1x/snl_norm1
x_train, x_vali, y_train, y_vali = train_test_split(selected_data_in1n, selected_data_out1n, test_size=0.2, shuffle=True)
x_train = x_train.reshape((x_train.shape[0], 24, 40, 1))
y_train = y_train.reshape((y_train.shape[0], 24, 40, 1))
x_vali = x_vali.reshape((x_vali.shape[0], 24, 40, 1))
y_vali = y_vali.reshape((y_vali.shape[0], 24, 40, 1))

## U-Net

In [ ]:
def double_conv_block(x, n_filters):
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    return f, p

def upsample_block(x, conv_features, n_filters):
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    x = layers.concatenate([x, conv_features])
    x = double_conv_block(x, n_filters)
    return x

def build_unet_model():
    inputs = layers.Input(shape=(24, 40, 1))
    f1, p1 = downsample_block(inputs, 32)
    f2, p2 = downsample_block(p1, 64)
    bottleneck = double_conv_block(p2, 128)
    u6 = upsample_block(bottleneck, f2, 64)
    u7 = upsample_block(u6, f1, 32)
    outputs = layers.Conv2D(1, 1, padding="same", activation="linear")(u7)
    model1 = tf.keras.Model(inputs, outputs, name="U-Net")
    return model1

model1 = build_unet_model()
model1.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse")

batch_size = 64
epochs = 200

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

history1 = model1.fit(
    x_train, y_train,
    validation_data=(x_vali, y_vali),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[early_stop]
)

## MLP

In [ ]:
def dense_block(x, n_units):
    x = layers.Dense(n_units, activation="relu", kernel_initializer="he_normal")(x)

    return x

def build_feedforward_model():
    inputs = layers.Input(shape=(24, 40, 1))

    x = layers.Flatten()(inputs)

    x = dense_block(x, 80)
    x = dense_block(x, 39)
    x = dense_block(x, 80)

    x = layers.Dense(24*40, activation="linear")(x)

    outputs = layers.Reshape((24, 40, 1))(x)
    model3 = tf.keras.Model(inputs, outputs, name="Feedforward-NN")
    return model3

model3 = build_feedforward_model()
model3.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse")

batch_size = 64
epochs = 200

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

history1 = model3.fit(
    x_train, y_train,
    validation_data=(x_vali, y_vali),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[early_stop]
)

## DIA as an additional input

In [ ]:
data_in1_trainsk = np.stack([data_in1_train, (data_in1_dia_train)], axis=-1)
data_in1_testsk = np.stack([data_in1_test, (data_in1_dia_test)], axis=-1)
np.random.seed(42)
random_indices = np.random.choice(data_in1_trainsk.shape[0], size=data_in1_trainsk.shape[0], replace=False)
selected_data_in1x = data_in1_trainsk[random_indices, :, :40]
selected_data_out1x = data_out1_train[random_indices, :, :40]
spec_norm1 = np.std(selected_data_in1x)
snl_norm1 = np.std(selected_data_out1x)
selected_data_in1n = selected_data_in1x/spec_norm1
selected_data_out1n = selected_data_out1x/snl_norm1
x_train, x_vali, y_train, y_vali = train_test_split(selected_data_in1n, selected_data_out1n, test_size=0.2, shuffle=True)
x_train = x_train.reshape((x_train.shape[0], 24, 40, 1))
y_train = y_train.reshape((y_train.shape[0], 24, 40, 1))
x_vali = x_vali.reshape((x_vali.shape[0], 24, 40, 1))
y_vali = y_vali.reshape((y_vali.shape[0], 24, 40, 1))

In [ ]:
def double_conv_block(x, n_filters):
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    return f, p

def upsample_block(x, conv_features, n_filters):
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    x = layers.concatenate([x, conv_features])
    x = double_conv_block(x, n_filters)
    return x

def build_unet_model():
    inputs = layers.Input(shape=(24, 40, 2))
    f1, p1 = downsample_block(inputs, 32)
    f2, p2 = downsample_block(p1, 64)
    bottleneck = double_conv_block(p2, 128)
    u6 = upsample_block(bottleneck, f2, 64)
    u7 = upsample_block(u6, f1, 32)
    outputs = layers.Conv2D(1, 1, padding="same", activation="linear")(u7)
    model1 = tf.keras.Model(inputs, outputs, name="U-Net")
    return model1

model1 = build_unet_model()
model1.compile(optimizer=tf.keras.optimizers.Adam(), loss="mse")

batch_size = 64
epochs = 200

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)
history1 = model1.fit(
    x_train, y_train,
    validation_data=(x_vali, y_vali),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[early_stop]
)

## Quantile loss function (optional)

In [ ]:
def quantile_loss(y_true, y_pred, q):
    e = y_true - y_pred
    return tf.keras.backend.maximum(q * e, (q - 1) * e)

def combined_mse_and_range_loss(y_true, y_pred, alpha=1.0, beta=1.0):
    y_true = tf.squeeze(y_true, axis=-1)
    mean = y_pred[..., 0]
    variation = tf.math.abs(y_pred[..., 1])
    mse_loss = tf.reduce_mean(tf.square(y_true - mean))

    y_lower = mean - variation
    y_upper = mean + variation
    loss_low = quantile_loss(y_true, y_lower, q=0.025)
    loss_up  = quantile_loss(y_true, y_upper, q=0.975)
    range_loss = tf.reduce_mean(loss_low + loss_up)

    return alpha * mse_loss + beta * range_loss


# Test dataset

In [ ]:
base_path = '/Users/jc/Documents/Snl4_data/Data_train/ERA5/Data/'
def load_and_reshape(name):
    arr = pd.read_csv(f'{base_path}{name}.csv', header=None).to_numpy()
    return arr.reshape(arr.shape[0], 40, 24).transpose(0, 2, 1)[:, :, :40]
data_input_test = load_and_reshape('input_test')
data_input_dia_test = load_and_reshape('input_dia_test')
data_output_test = load_and_reshape('output_test')
print(f"Test_size: {data_input_test.shape}")

data_input_testsk = data_input_test[..., None]
pred = model1.predict(data_input_testsk / spec_norm1) * snl_norm1